In [ ]:
import pandas as pd
import altair as alt

In [ ]:
#file = '/Users/ivan/Downloads/20240828_BARD1_AllScores.xlsx'
file = '/Users/ivan/Downloads/20240828_BARD1_AllScores_noX4.xlsx' #no X4A-K file

In [ ]:
def read_data(path):
    df = pd.read_excel(path)
    df = df[['target','Consequence','pos_id','snv_score_minmax']]

    filtered_consequences = ['missense_variant','synonymous_variant','stop_gained','stop_lost','stop_retained_variant']
    df = df.loc[df['Consequence'].isin(filtered_consequences)]

    index = []
    for i in range(len(df)):
        index.append(i)
    df = df.reset_index(drop = True)
    df.index = index
    
    return df

In [ ]:
def prep_data(df): #renames VEP consequence categories to be more human friendly
    
    df.loc[df['Consequence'] == 'missense_variant', 'Consequence'] = 'Missense'
    df.loc[df['Consequence'] == 'synonymous_variant', 'Consequence'] = 'Synonymous'
    #df.loc[df['Consequence'] == 'intron_variant', 'Consequence'] = 'Intron'
    df.loc[(df['Consequence'] == 'stop_gained') | (df['Consequence'] == 'stop_lost') | (df['Consequence'] == 'stop_retained_variant'), 'Consequence'] = 'Stop'
    #df.loc[(df['Consequence'] == 'splice_polypyrimidine_tract_variant') |(df['Consequence'] == 'splice_region_variant') | (df['Consequence'] == 'splice_acceptor_variant') | (df['Consequence'] == 'splice_donor_region_variant') | (df['Consequence'] == 'splice_donor_5th_base_variant') | (df['Consequence'] == 'splice_donor_variant'),'Consequence'] = 'Splice'
    #df.loc[df['Consequence'] == '3_prime_UTR_variant', 'Consequence'] = 'UTR'

    return df

In [ ]:
def rewrite_targets(df):
    
    i = 0
    while i < len(df):
        target = df['target'][i] #gets SGE target
        id = df['pos_id'][i]
        split = target.split("_") #splits SGE target name by _ and yields list with [Gene name, X(exon#)(region letter)]
        x_format = split[1] #gives just the exon and SGE target

        if x_format[1:3].isdigit(): #rewrites target for exon numbers greater than 9
            exon = int(x_format[1:3])
            df.loc[df['pos_id'] == id, 'target'] = exon
        elif x_format[1].isdigit(): #rewrites target for exon numbers less than 9
            exon = int(x_format[1])
            df.loc[df['pos_id'] == id, 'target'] = exon

        
        i += 1

    return df

In [ ]:
def exon_stats(df):

    exon_df_list = []
    grouped_exons = df.groupby(['target','Consequence'])
    for group_name, group_df in grouped_exons:
        exon, var_type = group_name
        scores = group_df['snv_score_minmax'].tolist()

        non = 0 #counters for nonfunctional group

        
        for elem in scores: #groups scores 
            if elem < (0.6):
                non += 1

        non_per = (non / len(group_df)) * 100
        same_per = (100 - non_per)
        exon_non_df = pd.DataFrame({'Exon': exon, 'Consequence': var_type, 'Function Type': 'Nonfunctional', 'Percent': non_per}, index = [0])
        exon_same_df = pd.DataFrame({'Exon': exon, 'Consequence': var_type, 'Function Type': 'Functional', 'Percent': same_per}, index = [0])
            
        exon_df = pd.concat([exon_non_df, exon_same_df])

        exon_df_list.append(exon_df)

    all_exons_df = pd.concat(exon_df_list)
    
    index = []
    for i in range(len(all_exons_df)):
        index.append(i)
    all_exons_df = all_exons_df.reset_index(drop = True)
    all_exons_df.index = index
    
    return all_exons_df

In [ ]:
def good_luck_graph(df):

    df = df.sort_values(by = ['Exon'])
    #print(df)
    chart = alt.Chart(df).mark_bar().encode(
            x = 'Exon:O',
            y = 'Percent',
            color = 'Function Type'
    ).facet(
        column = 'Consequence:N'
    )
    chart.show()

In [ ]:
def main():
    data = read_data(file)
    reannotated = prep_data(data)
    num_exons = rewrite_targets(data)
    all_stats = exon_stats(num_exons)
    good_luck_graph(all_stats)

In [ ]:
main()